# machine-learning-practica
### **Práctica ML** - Ejercicio de Bootcamp Inteligencia Artificial Full Stack Edición III

Este proyecto es un entregable para la práctica del Master Bootcamp Inteligencia Artificial Full Stack Edición III realizado por el centro de formación [@Keepcoding](https://github.com/KeepCoding)

---

El objetivo de la práctica es la predicción del precio del airbnb de los datos disponibles en el fichero [airbnb-listings-extract.csv](./airbnb-listings-extract.csv)

## Contenido

Los pasos esperados son los siguientes:
1. Preparación de datos: División train/test
2. Análisis exploratorio, por ejemplo:
    - Head, describe, dtypes, etc.
    - Outliers
    - Correlación
3. Preprocesamiento:
    - Eliminación de variables, mediante selección (random forest/Lasso), alta correlación, alto porcentaje de missings, o el método que se considere oportuno.
    - Generación de variables
5. Modelado:
    - Cross validation
    - Evaluación; mejor si lo hacéis de más de un modelo, porque así podéis comparar entre ellos.
6. Conclusión: escrita, no numérica; un par de líneas es más que suficiente.

## 1. Preparación de datos: División train/test

In [121]:
# Camenzamos con las librerías que usaremos
import numpy as np
import pandas as pd

# settings - descomentar a conveniencia
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

# sklearn imports
from sklearn.model_selection import train_test_split

#import matplotlib.pyplot as plt
#plt.style.use("seaborn-v0_8")

In [123]:
# Descargando los datos y primer contacto
datosABNB = pd.read_csv("./airbnb-listings-extract.csv",sep=";")
# aunque el punto 2 requiere en análisis exploratorio, dividir los datos me obliga mínimamente ver cual es la columna resultado
datosABNB.columns # parece que es Price

Index(['ID', 'Listing Url', 'Scrape ID', 'Last Scraped', 'Name', 'Summary',
       'Space', 'Description', 'Experiences Offered', 'Neighborhood Overview',
       'Notes', 'Transit', 'Access', 'Interaction', 'House Rules',
       'Thumbnail Url', 'Medium Url', 'Picture Url', 'XL Picture Url',
       'Host ID', 'Host URL', 'Host Name', 'Host Since', 'Host Location',
       'Host About', 'Host Response Time', 'Host Response Rate',
       'Host Acceptance Rate', 'Host Thumbnail Url', 'Host Picture Url',
       'Host Neighbourhood', 'Host Listings Count',
       'Host Total Listings Count', 'Host Verifications', 'Street',
       'Neighbourhood', 'Neighbourhood Cleansed',
       'Neighbourhood Group Cleansed', 'City', 'State', 'Zipcode', 'Market',
       'Smart Location', 'Country Code', 'Country', 'Latitude', 'Longitude',
       'Property Type', 'Room Type', 'Accommodates', 'Bathrooms', 'Bedrooms',
       'Beds', 'Bed Type', 'Amenities', 'Square Feet', 'Price', 'Weekly Price',
       'Month

In [124]:
# Ahora si separamos train/test
X1 = datosABNB.loc[:,datosABNB.columns != "Price"]
y1 = datosABNB["Price"]
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.3, shuffle=True, random_state=0)

## 2. Análisis exploratorio

In [128]:
# comencemos con shape, a ver que nos espera
X_train.shape # 88 columnas por explorar

(10346, 88)

In [254]:
# quiza hay alguna columna que todo sea null se podría quitar
areAllNulls = X_train.isnull().all()
areAllNulls[areAllNulls == True] # no hubo suerte

Series([], dtype: bool)

In [256]:
# Veamos las que contienen null
areSomeNulls = X_train.isnull().any()
areSomeNulls[areSomeNulls == True] # hay varias columnas con algún null, habrá que ver que tan útiles son

Name                              True
Summary                           True
Space                             True
Description                       True
Neighborhood Overview             True
Notes                             True
Transit                           True
Access                            True
Interaction                       True
House Rules                       True
Thumbnail Url                     True
Medium Url                        True
Picture Url                       True
XL Picture Url                    True
Host Name                         True
Host Since                        True
Host Location                     True
Host About                        True
Host Response Time                True
Host Response Rate                True
Host Acceptance Rate              True
Host Thumbnail Url                True
Host Picture Url                  True
Host Neighbourhood                True
Host Listings Count               True
Host Total Listings Count

In [270]:
# Veamos los datatypes
X_train.dtypes # muchos strings tenemos que ver que podemos obtener de ellos

ID                                  int64
Listing Url                        object
Scrape ID                           int64
Last Scraped                       object
Name                               object
Summary                            object
Space                              object
Description                        object
Experiences Offered                object
Neighborhood Overview              object
Notes                              object
Transit                            object
Access                             object
Interaction                        object
House Rules                        object
Thumbnail Url                      object
Medium Url                         object
Picture Url                        object
XL Picture Url                     object
Host ID                             int64
Host URL                           object
Host Name                          object
Host Since                         object
Host Location                     

In [268]:
# Un vistaso a todos
X_train.head(3)

,ID,Listing Url,Scrape ID,Last Scraped,Name,Summary,Space,Description,Experiences Offered,Neighborhood Overview,Notes,Transit,Access,Interaction,House Rules,Thumbnail Url,Medium Url,Picture Url,XL Picture Url,Host ID,Host URL,Host Name,Host Since,Host Location,Host About,Host Response Time,Host Response Rate,Host Acceptance Rate,Host Thumbnail Url,Host Picture Url,Host Neighbourhood,Host Listings Count,Host Total Listings Count,Host Verifications,Street,Neighbourhood,Neighbourhood Cleansed,Neighbourhood Group Cleansed,City,State,Zipcode,Market,Smart Location,Country Code,Country,Latitude,Longitude,Property Type,Room Type,Accommodates,Bathrooms,Bedrooms,Beds,Bed Type,Amenities,Square Feet,Weekly Price,Monthly Price,Security Deposit,Cleaning Fee,Guests Included,Extra People,Minimum Nights,Maximum Nights,Calendar Updated,Has Availability,Availability 30,Availability 60,Availability 90,Availability 365,Calendar last Scraped,Number of Reviews,First Review,Last Review,Review Scores Rating,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features
4706,2156319,https://www.airbnb.com/rooms/2156319,20170407214119,2017-04-08,GRANT VII Plaza Mayor,NaN,Beautiful and charming apartment recently deco...,Beautiful and charming apartment recently deco...,none,NaN,NaN,NaN,NaN,NaN,Rest hours: Monday to Friday from (phone numbe...,NaN,NaN,https://public.opendatasoft.com/api/v2/catalog...,NaN,1650712,https://www.airbnb.com/users/show/1650712,Ximena,2012-01-25,"Madrid, Madrid, Spain","I love to travel, discover new places and cult...",within an hour,100.0,NaN,https://a0.muscache.com/im/users/1650712/profi...,https://a0.muscache.com/im/users/1650712/profi...,La Latina,40.0,40.0,"email,phone,reviews,jumio","Sol, Madrid, Community of Madrid 28013, Spain",Sol,Sol,Centro,Madrid,Community of Madrid,28013,Madrid,"Madrid, Spain",ES,Spain,40.415418,-3.707123,Apartment,Entire home/apt,4,1.0,1.0,2.0,Real Bed,"TV,Internet,Wireless Internet,Air conditioning...",NaN,535.0,1800.0,150.0,40.0,2,20,2,1125,today,NaN,0,0,0,0,2017-04-08,9,2014-04-02,2014-10-20,93.0,10.0,9.0,9.0,9.0,10.0,9.0,NaN,NaN,strict,40.0,0.24,"40.4154180336,-3.70712273935","Host Has Profile Pic,Host Identity Verified,Is..."
6422,3377153,https://www.airbnb.com/rooms/3377153,20170407214119,2017-04-08,Nice flat in Plaza Mayor (lift),"Flat is in the centre town, really close to Pl...",You cannot find a better location to stay in M...,"Flat is in the centre town, really close to Pl...",none,"It´s really cool, with a lot of new business (...","Apartment has wifi, lift, wash-machine and air...","The best way is walking, since in Madrid all a...","The kitchen is available with oil, sugar, coff...",You can call me if you have any doubt or if yo...,Just enjoy Madrid! You are in your home in Sp...,https://a0.muscache.com/im/pictures/47295744/3...,https://a0.muscache.com/im/pictures/47295744/3...,https://public.opendatasoft.com/api/v2/catalog...,https://a0.muscache.com/im/pictures/47295744/3...,17037651,https://www.airbnb.com/users/show/17037651,Miguel,2014-06-20,"Madrid, Community of Madrid, Spain",I am love going out and meet new people from e...,within an hour,100.0,NaN,https://a0.muscache.com/im/users/17037651/prof...,https://a0.muscache.com/im/users/17037651/prof...,La Latina,1.0,1.0,"email,phone,reviews,manual_offline,jumio","La Latina, Madrid, Comunidad de Madrid 28005, ...",La Latina,Embajadores,Centro,Madrid,Comunidad de Madrid,28005,Madrid,"Madrid, Spain",ES,Spain,40.411131,-3.707258,Apartment,Entire home/apt,4,1.0,1.0,2.0,Real Bed,"Internet,Wireless Internet,Air conditioning,Ki...",NaN,299.0,1350.0,99.0,20.0,2,10,1,1125,today,NaN,4,16,33,298,2017-04-07,182,2014-08-09,2017-03-27,92.0,9.0,9.0,10.0,10.0,10.0,9.0,NaN,NaN,flexible,1.0,5.61,"40.411131472,-3.7072583983","Host Has Profile Pic,Host Identity V

In [260]:
#### detectemos Outliers!
# Analizamos un poco los datos
print(X_train.describe(include='all'))

                  ID                           Listing Url     Scrape ID  \
count   1.034600e+04                                 10346  1.034600e+04   
unique           NaN                                 10346           NaN   
top              NaN  https://www.airbnb.com/rooms/2156319           NaN   
freq             NaN                                     1           NaN   
mean    1.028354e+07                                   NaN  2.017038e+13   
std     5.555731e+06                                   NaN  5.374658e+08   
min     1.986400e+04                                   NaN  2.016010e+13   
25%     5.597253e+06                                   NaN  2.017041e+13   
50%     1.130555e+07                                   NaN  2.017041e+13   
75%     1.532606e+07                                   NaN  2.017041e+13   
max     1.858361e+07                                   NaN  2.017062e+13   

       Last Scraped                                Name  \
count         10346         